In [4]:
import os
from dotenv import load_dotenv
import asyncio

from autogen_agentchat.agents import AssistantAgent,UserProxyAgent
from autogen_agentchat.base import TaskResult,Handoff
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination,HandoffTermination

from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient


# Load environment variables from a .env file
load_dotenv("/home/azureuser/azure-ai-agent-workshop/.env")

True

In [5]:
model_client=AzureOpenAIChatCompletionClient(
            model=os.getenv("AZURE_OPENAI_MODEL"),
            azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
            api_key=os.getenv("AZURE_OPENAI_KEY"),
            azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT"),
            api_version=os.getenv("AZUER_OPENAI_API_VERSION")
        )

In [6]:
# define Agent
weather_agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    description="一个和用户交互的智能体。",
    system_message="不知道答案时，一定要转给用户。任务完成后回复<stop autogenworkshop>",
    handoffs=[Handoff(target="user", message="移交给用户。")],
)

In [7]:
# Define a termination condition. End the conversation if a specific text is mentioned.
text_termination = TextMentionTermination("<stop autogenworkshop>")
# Define a termination condition: end the conversation when the Agent sends a HandoffMessage
handoff_termination = HandoffTermination(target="user")
# Use the `|` operator to combine termination conditions, stopping the task if any condition is met
termination = handoff_termination | text_termination

In [8]:
# Define Team with type set to RoundRobinGroupChat
agent_team = RoundRobinGroupChat(participants=[weather_agent], termination_condition=termination, max_turns=None)


In [9]:
# Run the team and use the official Console tool to output in the proper format
stream = agent_team.run_stream(task="上海的天气如何?")
await Console(stream)

---------- user ----------
上海的天气如何?


---------- weather_agent ----------
[FunctionCall(id='call_TYfYaP9fJuqETLppuILtKu3g', arguments='{}', name='transfer_to_user')]
---------- weather_agent ----------
[FunctionExecutionResult(content='移交给用户。', call_id='call_TYfYaP9fJuqETLppuILtKu3g')]
---------- weather_agent ----------
移交给用户。


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='上海的天气如何?', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=64, completion_tokens=11), content=[FunctionCall(id='call_TYfYaP9fJuqETLppuILtKu3g', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='移交给用户。', call_id='call_TYfYaP9fJuqETLppuILtKu3g')], type='ToolCallExecutionEvent'), HandoffMessage(source='weather_agent', models_usage=None, target='user', content='移交给用户。', type='HandoffMessage')], stop_reason='Handoff to user from weather_agent detected.')

In [10]:
# Run the team and use the official Console tool to output in the proper format
stream = agent_team.run_stream(task="上海正在下雨,天气很糟糕。")
await Console(stream)

---------- user ----------
上海正在下雨,天气很糟糕。
---------- weather_agent ----------
天气不好时确实会影响心情。希望雨能快点停，天气变好！如果有其他需要了解的，欢迎随时问我。
---------- weather_agent ----------
[FunctionCall(id='call_MUKhWob8ID8JwhbGhBNY7A9e', arguments='{}', name='transfer_to_user')]
---------- weather_agent ----------
[FunctionExecutionResult(content='移交给用户。', call_id='call_MUKhWob8ID8JwhbGhBNY7A9e')]
---------- weather_agent ----------
移交给用户。


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='上海正在下雨,天气很糟糕。', type='TextMessage'), TextMessage(source='weather_agent', models_usage=RequestUsage(prompt_tokens=108, completion_tokens=35), content='天气不好时确实会影响心情。希望雨能快点停，天气变好！如果有其他需要了解的，欢迎随时问我。', type='TextMessage'), ToolCallRequestEvent(source='weather_agent', models_usage=RequestUsage(prompt_tokens=149, completion_tokens=11), content=[FunctionCall(id='call_MUKhWob8ID8JwhbGhBNY7A9e', arguments='{}', name='transfer_to_user')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='weather_agent', models_usage=None, content=[FunctionExecutionResult(content='移交给用户。', call_id='call_MUKhWob8ID8JwhbGhBNY7A9e')], type='ToolCallExecutionEvent'), HandoffMessage(source='weather_agent', models_usage=None, target='user', content='移交给用户。', type='HandoffMessage')], stop_reason='Handoff to user from weather_agent detected.')